In [4]:
#!pip install pyshp #pip install pyproj
#!pip install basemap
#!pip3 install cartopy
import os
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
#import pdb
#import scipy.spatial.qhull as qhull
#from scipy.interpolate import griddata
#import pyproj
#import pyshp
#from u_interpolate_small import regrid_irregular_quick
import u_interpolate_small as uint
#from datetime import date
from ndays import numOfDays
import glob
#import calendar
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf

In [5]:
# Define domain and time period
start_lat = 6 # -30 #37
end_lat = 12 # -17
start_lon = -4 # 15 # -4#15#-18
end_lon = 4 #30
start_year = '2010'
start_month = '07'#12'#12'
end_year = '2010'
end_month = '07'#12'#12'
start_day = '01'
end_day = '31'

In [ ]:
# find core indices using one file
core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/09/201009081045.nc'
core_data = nc.Dataset(core_filename,'r')
# read in cores lat/lon
lat_temp = np.array(core_data.variables['lat'][:])
lon_temp = np.array(core_data.variables['lon'][:])
#lon_filter = lon >-181 # acceptable values
#lon_fl = lon[lon_filter] #filtered long
lat_ind = np.where((lat_temp>=start_lat) & (lat_temp<=end_lat))[0]
lon_ind = np.where((lon_temp>= start_lon) & (lon_temp<=end_lon))[0]
lat = lat_temp[lat_ind[0]:lat_ind[-1]+1]
lon = lon_temp[lon_ind[0]:lon_ind[-1]+1]
#lon = lon_fl[(lon_fl>= start_lon) & (lon_fl<=end_lon)]
X, Y = np.meshgrid(lon,lat)

# find ndays
date1 = date(int(start_year), int(start_month), int(start_day))
date2 = date(int(end_year), int(end_month), int(end_day))
t = numOfDays(date1, date2)+1
#time = 
cores = np.zeros((len(lat),len(lon),t*24),dtype=float) #using every fourth value for hourly comparison- 24hr
tir = np.zeros((len(lat),len(lon),t*24),dtype=float) #using every fourth value for hourly comparison- 24hr
time_core = np.zeros((t*24)) #using every fourth value for hourly comparison- 24hr
#time = np.zeros((len(lat)-1,len(lon)-1,t*96))

In [ ]:
# find precip indices using one file
precip_filename = '/prj/global_water/MCS_Feng/global_v2/2d_fields/20100101.0000_20110101.0000/mcstrack_20100901_0630.nc'
precip_data = nc.Dataset(precip_filename,'r')
lat_p_temp = np.array(precip_data.variables['latitude'][:])
lon_p_temp = np.array(precip_data.variables['longitude'][:])
lat_ind_p = np.where((lat_p_temp[:,0]>=start_lat) & (lat_p_temp[:,0]<=end_lat))[0]
lon_ind_p = np.where((lon_p_temp[0,:]>= start_lon) & (lon_p_temp[0,:]<=end_lon))[0]
lat_p = lat_p_temp[lat_ind_p[0]:lat_ind_p[-1]+1,lon_ind_p[0]:lon_ind_p[-1]+1]
lon_p = lon_p_temp[lat_ind_p[0]:lat_ind_p[-1]+1,lon_ind_p[0]:lon_ind_p[-1]+1]
#X_p, Y_p = np.meshgrid(lon_p,lat_p)

In [ ]:
# read in precip data 
list_of_file_names_precip=[]

if int(start_year)==int(end_year): 
    dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+start_year+'0101.0000_'+str(int(start_year)+1)+'0101.0000'
    os.chdir(dir_name_precip)
    for m in range(int(start_month),int(end_month)+1,1):
            if m<10:
                month = str(m).zfill(2)
            else:
                month = str(m)   
            all_files=sorted(glob.glob("mcstrack_"+start_year+month+"*.nc"));
            #print(all_files)
            list_of_file_names_precip.append(all_files) # all days in month
             
elif int(start_year)<int(end_year):
    y=1
    for i in range(int(start_year),int(end_year)+1,1):
        dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
        #os.chdir(dir_name_precip)
        if y==1:
            for m in range(int(start_month),12+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m)  
                list_of_file_names_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                y=y+1            
        else:
            for m in range(1,int(end_month)+1,1):
                if m<10:
                    month = str(m).zfill(2)
                else:
                    month = str(m) 
                dir_name_precip =  '/prj/global_water/MCS_Feng/global_v2/2d_fields/'+str(i)+'0101.0000_'+str(i+1)+'0101.0000/'
                list_of_file_names_precip.append(sorted(glob.glob(dir_name_precip+'mcstrack_'+str(i)+month+'*.nc'))) # day 1
                    
    
    #print(dir_name_precip)
list_of_files_precip=[]                
for a in range(0,len(list_of_file_names_precip),1):
    list_of_files_precip = list_of_files_precip+list_of_file_names_precip[a]

In [ ]:
# preallocate
precip = np.zeros((len(lat_p[:,1]),len(lon_p[1,:]),t*24),dtype=float)
regidded_precip = np.zeros((len(lat),len(lon),t*24),dtype=float)
regidded_precip_test = np.zeros((len(lat),len(lon),t*24),dtype=float)
time_IMERG = np.zeros(t*24)

for l in range(0,len(list_of_files_precip),1):#22,31,1):#
    precip_filename = list_of_files_precip[l]
    print(precip_filename)
    print(l)
    precip_data = nc.Dataset(precip_filename,'r')
    precip_temp = np.array(precip_data.variables['precipitation'])[0,lat_ind_p[0]:lat_ind_p[-1]+1,lon_ind_p[0]:lon_ind_p[-1]+1]
    precip[:,:,l] = precip_temp  
    time_IMERG[l]= str(precip_filename[-16:-3])
    A = regrid_irregular_quick(lon_p[1,:],lat_p[:,1],lon,lat,precip_temp)
    #B = regrid_irregular_quick(lat_p[:,1],lon_p[1,:],lat,lon,precip_temp)
    regidded_precip[:,:,l] = A   
    #regidded_precip_test[:,:,l] = np.transpose(B)   


In [ ]:
# read in cores data
list_of_file_names = []

for i in range(0,len(time_IMERG),1):
    current_file = str(time_IMERG[i])#+'.nc'
    SY= current_file[0:4]
    MM= current_file[4:6]
    #DD= current_file[7:8]
    #HR= current_file[9:10]
    #MIN= current_file[11:12]
    all_file_names = '/prj/Africa_cloud/ch9_wavelet/'+SY+'/'+MM+'/'+current_file[0:-2]+'.nc' 
    #print(all_file_names)
    if os.path.isfile(all_file_names):      
    #all_file_names= dir_name+ #sorted(glob.glob(dir_name+start_year+month+"*30.nc"));
        list_of_file_names.append(all_file_names) # all days in month
    else:
        list_of_file_names.append('file_not_available') # all days in month
            
 
#print(list_of_file_names)

In [ ]:
#counter = 0
for l in range(0,len(list_of_file_names),1):#2, min(t*96,len(list_of_files)), 4):#range(len(list_of_files)): # 96 files per day
    core_filename = list_of_file_names[l]#'/prj/Africa_cloud/ch9_wavelet/'+str(i)+'/'+month+'/'  list_of_files[l]
    print(core_filename)
    if os.path.isfile(all_file_names):      
        core_data = nc.Dataset(core_filename,'r')
        print(l)
        cores_temp = np.array(core_data.variables['cores'])[0,lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]
        cores[:,:,l] = cores_temp   
        tir_temp = np.array(core_data.variables['tir'])[0,lat_ind[0]:lat_ind[-1]+1,lon_ind[0]:lon_ind[-1]+1]
        tir[:,:,l] = tir_temp   
#    df['Column Name'] = pd.to_datetime(df['Column Name'])
        time_core[l] = (core_filename[-15:-3])
        core_data.close()  
    else:
        cores[:,:,l] = cores_temp*np.nan  
        tir[:,:,l] = tir_temp*np.nan   
        time_core[l] = (core_filename[-15:-3])
  

In [ ]:
print(cores.shape)
print(precip.shape)
print(regidded_precip.shape)

In [ ]:
# visualize core data at inherent grid/resol
day = 24*1
plt.figure(figsize=(10,6))
plt.contour(X,Y,cores[:,:,day],cmap='Purples',linewidths=2)
plt.contourf(X,Y,tir[:,:,day],cmap='winter')
#plt.clim([-80, 80])
plt.colorbar(label='TIR [Tb oC]')#plt.contour(X,Y,cores[:,:,22],color='red')
#plt.colorbar(label='power')
#ax = plt.axes(projection = ccrs.Mercator())
# This will add borders
#ax.add_feature(cf.BORDERS)
plt.title('Core contour in purple')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim([X[0,0],X[0,-1]])
plt.ylim([Y[0,0],Y[-1,0]])
plt.show()

# visualize core data at inherent grid/resol
plt.figure(figsize=(10,6))
plt.contour(X,Y,cores[:,:,day],cmap='Purples',linewidths=2)
plt.contourf(lon_p,lat_p,precip[:,:,day],cmap='jet')#
plt.colorbar(label='IMERG original precip [mm/hr]')
plt.title('Core contour in purple')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim([X[0,0],X[0,-1]])
plt.ylim([Y[0,0],Y[-1,0]])
plt.show()

# visualize core data at inherent grid/resol
plt.figure(figsize=(10,6))
plt.contour(X,Y,cores[:,:,day],cmap='Purples',linewidths=2)
plt.contourf(X,Y,regidded_precip[:,:,day],cmap='jet') #regidded_precip
plt.colorbar(label='regridded precip [mm/hr]')
plt.title('Core contour in purple')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim([X[0,0],X[0,-1]])
plt.ylim([Y[0,0],Y[-1,0]])
plt.show()

# visualize tir and imerg data 
plt.figure(figsize=(10,6))
plt.contour(X,Y,regidded_precip[:,:,day],levels=[1, 5, 10, 15, 20],cmap='jet',linewidths=2) #regidded_precip
plt.contourf(X,Y,tir[:,:,day],cmap='winter')
plt.colorbar(label='TIR [Tb oC]')#plt.contour(X,Y,cores[:,:,22],color='red')
plt.title('Regridded rainfall contour')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim([X[0,0],X[0,-1]])
plt.ylim([Y[0,0],Y[-1,0]])
plt.show()

In [ ]:
# find rainfall pixels collocated with cores
cores_1D = cores.flatten()
precip_1D = regidded_precip.flatten() # cores and precip have same size and indexing - obs in connis notebook
cores_ind_gt_0 = np.where(cores_1D>0)[0]
precip_collocated_w_cores = precip_1D[cores_ind_gt_0] #dat in connis notebook

cores_ind_gt_0
print(precip_collocated_w_cores.shape)
print(np.nanmax(precip_collocated_w_cores))

In [ ]:
print(np.nanmax(precip_1D))

In [ ]:
# calculate frequency of rainfall above a threshold 
# dat = 
# threshold = 30mm/hr

def calc_treshReached(dat, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat))

In [ ]:
# calculate frequency of rainfall above a threshold relative to the same frequency for the full rainfall field (native)

def calc_treshReached_vsObs(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.isfinite(dat)) / np.sum(np.array(obs)>thresh)/np.sum(np.isfinite(obs))

In [ ]:
# calculate fraction of caught rainfall pixels above a threshold
# calc_treshMissed in connis script

def calc_treshcaught(dat, obs, thresh):
    return np.sum(np.array(dat)>thresh)/np.sum(np.array(obs)>thresh)

In [ ]:
# calculate frequency of rainfall above a threshold for regions with wavelet power above a given percentile

def percentile(dat, wav, thresh, perc):
    pval = np.percentile(wav, perc)
    dats = np.array(dat)[np.array(wav)>=pval]
    return np.sum(dats>thresh)/np.sum(np.isfinite(dats))

In [ ]:
# fraction of pixels where threshold exceeded
# dat = precip_collocated_w_cores
# obs = precip_1D
thresh = 20
# treached = calc_treshReached(precip_collocated_w_cores,thresh)
# print(treached)

In [ ]:
# fraction of caught pixels
# tcaught = calc_treshcaught(precip_collocated_w_cores,precip_1D, thresh)
# print(tcaught)

In [ ]:
# fraction of threshold reached in dat vs threshold reached in obs
tfrac = calc_treshReached_vsObs(precip_collocated_w_cores,precip_1D, thresh)
print(tfrac)

In [ ]:
# fraction of rainfall pixels reached above intensity threshold to all rainfall pixels (collocated with cores)
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
treached1 = []

for intens in range(5,50,5):   
    cc = calc_treshReached(precip_collocated_w_cores, intens)
    cc1 = calc_treshReached(precip_1D, intens)
    treached.append(cc)
    treached1.append(cc1)

ax.plot(range(5,50,5), treached,color='green',label='Precip collocated w/ cores')
ax.plot(range(5,50,5), treached1,color='red',label='IMERG precip')

ax.set_title("Intensity probability (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
plt.legend()

In [ ]:
# fraction of rainfall pixels reached above intensity threshold to all rainfall pixels (collocated with cores)precip
precip_ind_gt_1 = np.where(precip_collocated_w_cores>1)[0] #find pixels with > 1mm precip
precip_collocated_w_cores_gt_1 = precip_collocated_w_cores[precip_ind_gt_1] #dat in connis notebook
precip_ind_gt_1 = np.where(precip_1D>1)[0] #find pixels with > 1mm precip
precip_1D_gt_1 = precip_1D[precip_ind_gt_1] 


f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
treached1 = []

for intens in range(5,50,5):   
    cc = calc_treshReached(precip_collocated_w_cores_gt_1, intens)
    cc1 = calc_treshReached(precip_1D_gt_1, intens)
    treached.append(cc)
    treached1.append(cc1)

ax.plot(range(5,50,5), treached,color='green',label='Precip collocated w/ cores')
ax.plot(range(5,50,5), treached1,color='red',label='IMERG precip')

ax.set_title("Precip>1mm only- Intensity probability (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
plt.legend()

In [ ]:
# plot frequency of caught pixels with rainfall above a threshold for regions with wavelet power above a given 
# percentile
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

treached = []
for intens in range(5,50,5):   
    cc = percentile(precip_collocated_w_cores, cores_1D[cores_ind_gt_0], intens, 75)
    treached.append(cc)
    #plotdic.append((treached,dat[1]))
#for pp in plotdic:    
#ax.plot(range(5,50,5), pp[0], label=pp[1])

ax.plot(range(5,50,5), treached,color='green')
ax.set_title("Intensity probability for 75th centile power (%) (x[treshhold]/x)")
plt.xlabel('intensity of rainfall [mm/hr]')
#ax.ylabel('fraction of rainfall pixels caught above intensity threshold that have >= 75th percentile wavelet power')
#plt.legend()
#f.savefig("/home/ck/DIR/cornkle/data/nflics/core_tests/"+'rainP_75p_wpower.jpg')

In [ ]:
# fraction of rainfall pixels caught above intensity threshold to all caught rainfall pixels
f = plt.figure(figsize=(6,4), dpi=200)
ax = f.add_subplot(111)

tcaught = []
for intens in range(5,50,5):   
    cc = calc_treshcaught(precip_collocated_w_cores_gt_1, precip_1D_gt_1, intens)
#    cc = calc_treshcaught(precip_collocated_w_cores, precip_1D, intens)
    tcaught.append(cc)

ax.plot(range(5,50,5), tcaught,color='red')
ax.set_title("Retained pixels per intensity (%)")
plt.xlabel('intensity of rainfall [mm/hr]')
#f.savefig("/home/ck/DIR/cornkle/data/nflics/core_tests/"+'_missedCases_southernAfricaDomain.jpg')   

In [ ]:
# read cores daily data
#core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/06/201006081045.nc'
#core_filename = '/prj/Africa_cloud/ch9_wavelet/2010/',month,'/201006081045.nc'
#core_data = nc.Dataset(core_filename,'r')
#print(core_data)
#cores_temp = np.array(core_data.variables['cores'])
#cores = np.array(core_data.variables['cores'])
#time = core_data.variables['time'][:]

#cores = cores.append(cores_temp)
# make a lat x lon x time dataset for monthly cores
#global_attributes = core_data.__dict__


# read precip data
#precip_filename = '/prj/global_water/MCS_Feng/global_v2/2d_fields/'
#precip_data = nc.Dataset(precip_filename,'r')

# visualize core data at inherent grid/resol
#plt.figure(figsize=(10,6))
#plt.contourf(X,Y,np.max(cores,axis=2),cmap='jet')
#plt.colorbar(label='power')
#plt.title(cores.time)
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.show()

# visualize regridded data
# plt.figure(figsize=(10,6))
#plt.contourf(X_new,Y_new,A,cmap='jet')
#plt.colorbar(label='power')
#plt.title(cores.time)
#plt.xlabel('Longitude')
#plt.ylabel('Latitude')
#plt.show()

In [ ]:
# replace with precip data grid - bring coarse precip data (0.1) to fine core grid (0.04)
#new_lat = np.linspace(min(lat),max(lat),2000)#(lat[2]-lat[1])*2)
#new_lon = np.linspace(min(lon),max(lon),1900)#(lat[2]-lat[1])*2)
#X_new, Y_new = np.meshgrid(new_lon,new_lat)
#A = regrid_irregular_quick(lat_p,lon_p,lon,lat,cores_temp)
#regidded_precip[:,:,l] = A    

In [ ]:
#df = pd.DataFrame()
# convert to datetime using pd.to_datetime
#df['date'] = pd.to_datetime(df['date'])
#da = xr.DataArray(data=cores,coords={"lat":(["x","y"],lat),
 #                                    "lon":(["x","y"],lon),
  #                                   "day":str(time_core)},
   #                                 dims=["x","y","day"])
